In [1]:
# Jupyter에서 중첩 이벤트 루프 오류 방지 (asyncio 충돌 해결)
import nest_asyncio
nest_asyncio.apply()

In [2]:
# 환경 변수 및 경로 설정
from dotenv import load_dotenv, find_dotenv
import os
import asyncio
from pathlib import Path
from pyzerox import zerox

load_dotenv(find_dotenv())  # .env 파일 로드
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not set. Check your .env file.")

# PDF 추출용 poppler 바이너리 경로 (Windows 등)
poppler_path = os.getenv("POPPLER_PATH")
if poppler_path:
    poppler_dir = Path(poppler_path)
    if poppler_dir.exists():
        path_env = os.environ.get("PATH", "")
        if str(poppler_dir) not in path_env.split(os.pathsep):
            # 시스템 PATH에 poppler 경로가 없다면 추가
            os.environ["PATH"] = f"{poppler_dir}{os.pathsep}{path_env}" if path_env else str(poppler_dir)
    else:
        raise RuntimeError(f"POPPLER_PATH not found: {poppler_dir}")


In [ ]:
# 문서 변환 실행 설정 및 배치 처리
model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")  # 사용할 OpenAI 모델
input_dir = Path("./documents_with_english_titles")  # 입력 문서 폴더
output_dir = Path("./output")  # 결과 저장 폴더
output_dir.mkdir(parents=True, exist_ok=True)  # 폴더 생성

async def main():
    results = []  # 파일별 처리 결과 저장
    for file_path in sorted(input_dir.glob('*')):
        if not file_path.is_file():
            continue  # 하위 폴더 등은 건너뜀

        # pyzerox를 이용해 문서를 Markdown 등으로 변환
        result = await zerox(
            file_path=str(file_path),
            model=model,
            output_dir=str(output_dir),
            custom_system_prompt=None,
            select_pages=None,
        )

        print(f"{file_path} is done")  # 진행상황 로그
        results.append(result)

    return results

results = asyncio.run(main())  # 비동기 실행 시작

documents_with_english_titles\delegation_of_authority.pdf is done
documents_with_english_titles\employee_benefits_and_welfare_faq.pdf is done
documents_with_english_titles\employee_benefits_and_welfare_guide.pdf is done
documents_with_english_titles\employee_handbook_and_hr_policy.pdf is done
documents_with_english_titles\expense_management_guide.pdf is done
documents_with_english_titles\it_support_guide.pdf is done
documents_with_english_titles\legal_and_compliance_policy.pdf is done
